# Exploracion Inicial de los datos.

## En este notebook vamos a ver que datos tenemos, su tipo, sus valores nulos, si hay que unificarlos, etc.

## indice
- [Importamos librerias y configuraciones](#importamos-librerias-y-configuraciones)
- [Importamos Datos](#importamos-datos)
- [Comprobamos longitud de los DataFrames](#comprobamos-longitud-de-los-dataframes)
- [Union DataFrames](#union-dataframes)
- [Sustitución Valores Binarios (Yes/No, N/Y).](#sustitución-valores-binarios-(yes/no,-n/y).)
- [Reducción de datos](#reduccion-de-datos)
- [Conversion Tipo Datos](#conversion-tipo-datos)
- [Salvamos Datos](#salvamos-datos)

---
---

## Importamos librerias y configuraciones

In [41]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
---

## Importamos Datos

In [42]:
df_emplo_survey_data = pd.read_csv('../../datos/employee_survey_data.csv').reset_index(drop=True)
df_emplo_survey_data.sample()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
3839,3840,3.00,3.00,3.00


In [43]:
df_general_data = pd.read_csv('../../datos/general_data.csv').reset_index(drop=True)
df_general_data.sample()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
3995,29,Yes,Travel_Rarely,Research & Development,8,2,Medical,1,3996,Male,1,Manager,Divorced,28530,4.00,Y,14,8,1,4.00,5,2,2,0


In [44]:
df_mana_survey_data = pd.read_csv('../../datos/manager_survey_data.csv').reset_index(drop=True)
df_mana_survey_data.sample()

,EmployeeID,JobInvolvement,PerformanceRating
2702,2703,2,3


---
---

## Comprobamos longitud de los DataFrames

#### (comprobamos que los 3dfs tengan el mismo número de filas)

In [45]:
len_dfs = [df_general_data.shape[0], df_emplo_survey_data.shape[0], df_mana_survey_data.shape[0]]
set(len_dfs)

{4410}

#### como podemos ver, todas tienen 4410 filas, ya que si no, al convertir la lista en un set y por defecto, eliminar los valores repetidos, en caso de haber longitudes diferentes, nos saldrían más de un unico valor.

---
---

## Union DataFrames

#### Ahora que hemos visto que tenemos el mismo número de filas, vamos a unir todos los datos en un dataframe mediante el ```EmployeeID``` [también nos hemos asegurado que EmployeeID tiene el mismo nombre en los 3 dfs]

#### cambiamos los nombres de las columnas del df ```df_mana_survey_data``` para que luego sepamos que pertenecen a la opinión del manager sobre ese empleado.

In [46]:
df_mana_survey_data.columns = ['EmployeeID', 'Manager_opinion_JobInvolvement', 'Manager_opinion_PerformanceRating']
df_mana_survey_data.sample()

,EmployeeID,Manager_opinion_JobInvolvement,Manager_opinion_PerformanceRating
2655,2656,2,4


In [47]:
df = df_emplo_survey_data.merge(df_general_data, on='EmployeeID').merge(df_mana_survey_data, on='EmployeeID')
df.sample()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Manager_opinion_JobInvolvement,Manager_opinion_PerformanceRating
3354,3355,3.00,3.00,3.00,42,No,Travel_Rarely,Sales,11,1,Technical Degree,1,Male,2,Human Resources,Single,59400,7.00,Y,16,8,0,10.00,2,5,0,3,2,3


#### comprobamos si tenemos duplicados

In [48]:
df.duplicated().sum()

0

---
---

## Sustitución Valores Binarios (Yes/No, N/Y).

#### Vamos a sustituir los valores (Yes/No, N/Y) por Booleanos True y False.

#### Variables a modificar: 
- Attrition
- Over18

In [ ]:
df['Attrition'].unique()

array(['No', 'Yes'], dtype=object)

In [ ]:
df['Over18'].unique()

array(['Y'], dtype=object)

In [ ]:
df['Over18'].value_counts()

Over18
Y    4410
Name: count, dtype: int64

#### hemos visto que ```Over18``` tiene un 100 de 'Y' como valor [lo cual tiene sentido], por lo que la vamos a eliminar de nuestro df

In [ ]:
df.drop(columns=['Over18'], inplace=True)

df['Attrition'] = df['Attrition'].map({'No': False, 'Yes': True})

In [ ]:
df.sample(3)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Manager_opinion_JobInvolvement,Manager_opinion_PerformanceRating
552,553,1.00,3.00,3.00,46,False,Travel_Rarely,Research & Development,6,4,Medical,1,Male,3,Sales Executive,Married,21740,3.00,11,8,1,23.00,3,12,4,9,3,3
855,856,4.00,1.00,2.00,32,False,Non-Travel,Research & Development,10,4,Life Sciences,1,Female,5,Research Scientist,Married,23800,4.00,13,8,0,9.00,5,6,1,2,3,3
3457,3458,3.00,4.00,3.00,43,False,Travel_Rarely,Sales,25,2,Life Sciences,1,Male,3,Research Scientist,Divorced,50550,5.00,22,8,0,18.00,2,1,0,0,2,4


---
---

## Conversion Tipo Datos

#### Vamos a comprobar el tipo de dato que tienen nuestras varibales junto a su valor y en caso de que sea necesario, cambiarlo, eliminar la columna, etc.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EmployeeID                         4410 non-null   int64  
 1   EnvironmentSatisfaction            4385 non-null   float64
 2   JobSatisfaction                    4390 non-null   float64
 3   WorkLifeBalance                    4372 non-null   float64
 4   Age                                4410 non-null   int64  
 5   Attrition                          4410 non-null   bool   
 6   BusinessTravel                     4410 non-null   object 
 7   Department                         4410 non-null   object 
 8   DistanceFromHome                   4410 non-null   int64  
 9   Education                          4410 non-null   int64  
 10  EducationField                     4410 non-null   object 
 11  EmployeeCount                      4410 non-null   int64

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
EmployeeID,4410.00,2205.50,1273.20,1.00,1103.25,2205.50,3307.75,4410.00
EnvironmentSatisfaction,4385.00,2.72,1.09,1.00,2.00,3.00,4.00,4.00
JobSatisfaction,4390.00,2.73,1.10,1.00,2.00,3.00,4.00,4.00
WorkLifeBalance,4372.00,2.76,0.71,1.00,2.00,3.00,3.00,4.00
Age,4410.00,36.92,9.13,18.00,30.00,36.00,43.00,60.00
DistanceFromHome,4410.00,9.19,8.11,1.00,2.00,7.00,14.00,29.00
Education,4410.00,2.91,1.02,1.00,2.00,3.00,4.00,5.00
EmployeeCount,4410.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00
JobLevel,4410.00,2.06,1.11,1.00,1.00,2.00,3.00,5.00
MonthlyIncome,4410.00,65029.31,47068.89,10090.00,29110.00,49190.00,83800.00,199990.00


#### **Cambios que vamos a hacer:**

##### **Eliminación de columnas**

- ```EmployeeID``` ya que para entrenar a nuestro modelo predictivo no nos aporta información. Más adelante para otro tipo de estudio si nos puede ser útil.

- ```EmployeeCount``` ya que todos los registros contienen el valor de 1.

- ```StandardHours``` ya que todos los registros contienen el valor de 8 horas.

guardamos los datos para otro posible estudio ya que lo tenemos todo unido y con los tipos de datos correctos.

In [ ]:
df.to_pickle('../../datos/futuro_estudio/df_employee_data.pkl')

Procedemos con la eliminación de columnas

In [ ]:
df.drop(columns=['EmployeeID', 'EmployeeCount', 'StandardHours'], inplace=True)
df.sample()

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Manager_opinion_JobInvolvement,Manager_opinion_PerformanceRating
710,1.00,1.00,2.00,36,False,Travel_Frequently,Sales,16,2,Life Sciences,Male,1,Research Scientist,Married,12810,2.00,11,0,17.00,2,15,6,13,3,3


**Cambio de Valores (categorización de las variables numericas)**

#### Casi todos los datos están con su tipo correcto, además, hemos visto que al cambiar los valores de ```Attrition``` ha obtenido el tipo de dato booleno. Solo nos quedaría:

- Cambiar valores de ```Education```: le asignaremos su valor original ya que nos viene como numero y así poder ver si tienen orden o no una vez tengamos su valores categóricos.

In [ ]:
df['Education'] = df['Education'].map({
                                        1: 'Below College',
                                        2: 'College',
                                        3: 'Bachelor',
                                        4: 'Master',
                                        5: 'Doctor'
                                    })

df['Education'].unique()

array(['College', 'Below College', 'Master', 'Doctor', 'Bachelor'],
      dtype=object)

---
---

## Salvamos Datos

#### Una vez realizada una exploraci'on